In [ ]:
import os
from langchain_community.vectorstores import Chroma
#OpenAIEmbeddings는 비용나감 / llm모델이랑 별개의 작업이므로 다른 모델 사용 가능
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate

In [ ]:
#임베딩 모델
model_name='jhgan/ko-sroberta-nli'
model_kwargs = {"device":"cpu"}
encode_kwargs={'normalize_embeddings':True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs

)

In [ ]:
# 저장할 경로 지정
DB_PATH = "./chroma_db"

# 문서를 디스크에 저장. 저장시 persist_directory에 저장할 경로를 지정합니다.
vector_db = Chroma.from_documents(splits, embeddings, persist_directory=DB_PATH)


In [ ]:
vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

앞서 저장된 vector_db 불러오기

In [ ]:
#openai 인증키 설정
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
#프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant.
            You are a chatbot that provides information about pet-friendly facilities.
            Please respond based on the context.
            Answer questions using only the following context.
            모든 정보를 알려줘. 간결하게 답해줘.
            If you don't know the answer just say you don't know,
            don't make it up:
            \n\n
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

In [ ]:
# llm 모델
llm = ChatOpenAI(model="gpt-4o",temperature=0)

In [ ]:
# retriever 설정
retriever = MultiQueryRetriever.from_llm(
    retriever=vector_db.as_retriever(
    search_type='mmr',
    search_kwargs={'k':3, 'fetch_k': 3}),
    llm=llm
)

In [ ]:
#langchain
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer'),
        get_chat_history=lambda h: h,
        return_source_documents=True,
        verbose = True,
        combine_docs_chain_kwargs={"prompt": prompt}
    )

In [ ]:
# 질문 입력
query="은평구에 있는 반려동물 동반 가능 카페를 알려줘"
answer=conversation_chain(query)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
            You are a helpful assistant. 
            You are a chatbot that provides information about pet-friendly facilities. 
            Please respond based on the context.
            Answer questions using only the following context.
            간결하게 답해줘.
            If you don't know the answer just say you don't know, 
            don't make it up:
            


            개상전 애견카페은(는) (03397) 서울특별시 은평구 통일로 727-5에 위치한 카페 시설입니다.
운영시간 : 매일 11:00~21:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 없음.
입장(이용료)가격 정보 : 변동 | 애견 동반 추가 요금 : 없음.
좌표 : N37.61021895, E126.929202.
추가적인 정보는 https://www.instagram.com/gaesangjeon/?utm_source=ig_profile_share&igshid=p7pa7v4n8y9z에서 확인하세요.

쿠쿠펫 애견카페은(는) (34190) 대전광역시 유성구 봉명로 27-7에 위치한 카페 시설입니다.
운영시간 : 매일 11:00~20:00 | 휴무일 : 연중무휴 | 주차가능 | 반려동물 동반가능 | 반려동물 제한사항 : 매너벨트, 안거나 목줄 착용.
입장(이용료)가격 정보 : 12000원 | 애견 동반 추가 요금 : 5,000원.
좌표 : N3